In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
dataset = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/com_lga.csv')

In [4]:
dataset.shape

(8757, 113)

In [5]:
dataset.head(3)

,Date,Hour,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,...,4.1_lag_3,4.2_lag_3,4.3_lag_3,4.4_lag_3,4.5_lag_3,5.0_lag_3,5.1_lag_3,5.2_lag_3,5.3_lag_3,arrival_lag_3
0,2018-01-01,3,0,0,0,1,0,0,0,0,...,11.0,8.0,7.0,0.0,10.0,0.0,1.0,2.0,0.0,3.0
1,2018-01-01,4,2,0,0,0,0,0,0,0,...,2.0,3.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,2018-01-01,5,2,0,0,0,0,0,1,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
lag_columns = [c for c in dataset.columns if 'lag' in c]
len(lag_columns)

75

In [7]:
dataset = dataset[[c for c in dataset.columns if c not in lag_columns]]
dataset.shape

(8757, 38)

In [8]:
DateColumns = ['Date']

ext_columns = ['Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

targetColumns = [c for c in dataset.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']
len(targetColumns)

24

In [9]:
features_cols = [c for c in dataset.columns if c not in targetColumns and c not in DateColumns]
len(features_cols)

13

In [10]:
sep = int(0.75*len(dataset))
print(sep)


trainData = dataset[:sep]
testData = dataset[sep:]

print(trainData.shape)
print(testData.shape)

6567
(6567, 38)
(2190, 38)


In [11]:
X_train = trainData[features_cols].values
X_train = torch.tensor(X_train).float().to(device)
print(X_train.shape)

y_train = trainData[targetColumns].values
y_train = torch.tensor(y_train).float().to(device)
print(y_train.shape)

X_test = testData[features_cols].values
X_test = torch.tensor(X_test).float().to(device)
print(X_test.shape)

y_test = testData[targetColumns].values
y_test = torch.tensor(y_test).float().to(device)
print(y_test.shape)

torch.Size([6567, 13])
torch.Size([6567, 24])
torch.Size([2190, 13])
torch.Size([2190, 24])


In [12]:
def create_inout_sequences(x,y, tw):
    inout_seq = []
    L = len(x)
    for i in range(L-tw):
        train_seq_x = x[i:i+tw]
        train_seq_y = y[i:i+tw]
        train_seq = torch.cat((train_seq_x,train_seq_y),axis=1)
        
#         train_label = y[i+tw:i+tw+1]
        train_label = y[i+1:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [13]:
bptt = 6

In [14]:
train_inout_seq = create_inout_sequences(X_train,y_train, bptt)

In [15]:
train_inout_seq[0][0].shape,train_inout_seq[0][1].shape

(torch.Size([6, 37]), torch.Size([6, 24]))

In [16]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1, layers=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size, num_layers=layers)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(layers,1,self.hidden_layer_size),
                            torch.zeros(layers,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
#         return predictions[-1]
        return predictions

In [17]:
layers = 2
model = LSTM(input_size = 37, hidden_layer_size=100, output_size=24, layers=layers).to(device)
loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:
epochs = 100


for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(layers, 1, model.hidden_layer_size).to(device),
                        torch.zeros(layers, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 6.74877930
epoch:  26 loss: 5.46323299
epoch:  51 loss: 4.93113947
epoch:  76 loss: 5.70503044
epoch:  99 loss: 5.3055567741


In [19]:
test_inout_seq = create_inout_sequences(X_test,y_test, bptt)

In [20]:
model.eval()
prediction = []
with torch.no_grad():
    for seq, labels in test_inout_seq:
        model.hidden = (torch.zeros(layers, 1, model.hidden_layer_size),
                        torch.zeros(layers, 1, model.hidden_layer_size))
        prediction.append(model(seq)[-1])

In [21]:
y_test_ = torch.stack([labels[-1] for seq, labels in test_inout_seq], axis=0).detach().cpu().numpy()
y_pred_ = torch.stack(prediction).detach().cpu().numpy()

In [22]:
y_test_.shape, y_pred_.shape

((2184, 24), (2184, 24))

In [23]:
r2_score(y_test_, y_pred_, multioutput='variance_weighted')

0.8344461378959372

## edge prediction

In [24]:
edge_prediction_df = pd.DataFrame(y_pred_)
edge_prediction_df.columns = testData[6:][targetColumns].columns
edge_prediction_df.head(2)

,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,2.1,2.2,...,4.0,4.1,4.2,4.3,4.4,4.5,5.0,5.1,5.2,5.3
0,91.468262,14.692745,47.516056,14.249146,23.561430,6.257653,31.347113,5.476247,6.273333,13.835718,...,0.009400,15.948349,14.658237,21.535744,0.124913,28.599487,-0.046715,0.036951,0.019754,0.004486
1,0.594693,0.313245,2.800231,0.997516,1.020387,0.549432,2.358328,-0.014641,0.119129,1.200514,...,0.001455,0.371059,0.207977,0.782925,0.101534,0.344870,-0.008788,-0.005159,0.002701,0.009000


In [25]:
zone_weights = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/com_lga_weights.csv')

In [26]:
edge_prediction_df.shape

(2184, 24)

In [27]:
zone_weights['Borough'] = zone_weights['Borough'].astype(str)

In [28]:
zone_weights['Borough']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
252    5.2
253    5.2
254    5.3
255    5.3
256    5.3
Name: Borough, Length: 257, dtype: object

In [29]:
edge_prediction_df.columns

Index(['0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0', '2.1', '2.2',
       '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3', '4.4', '4.5',
       '5.0', '5.1', '5.2', '5.3'],
      dtype='object')

In [30]:
boroughs = list(edge_prediction_df.columns)
for bor in boroughs:
    print(bor)
    
    weight_df = zone_weights[zone_weights.Borough == bor]
    
    print(len(weight_df.DOLocationID))
    
    for b_zone,z_weight in zip(weight_df.DOLocationID.values,weight_df.zone_weight.values):        
        edge_prediction_df[b_zone] = edge_prediction_df[bor] * z_weight

0.0
14
0.1
4
0.2
21
1.0
15
1.1
17
1.2
20
1.3
10
2.0
14
2.1
19
2.2
8
2.3
10
3.0
6
3.1
4
3.2
8
4.0
1
4.1
18
4.2
23
4.3
12
4.4
5
4.5
9
5.0
4
5.1
8
5.2
4
5.3
3


In [31]:
edge_prediction_df.shape

(2184, 281)

In [32]:
select_cols = [c for c in edge_prediction_df.columns if c not in boroughs]
edge_prediction_df = edge_prediction_df[select_cols]
edge_prediction_df.shape

(2184, 257)

In [33]:
edge_testData = pd.read_csv('/home/urwa/Documents/side_projects/urban/data/featureData/lga.csv')
edge_testData = edge_testData[sep+bptt:]

lag_columns = [c for c in edge_testData.columns if 'lag' in c]
edge_testData = edge_testData[[c for c in edge_testData.columns if c not in lag_columns]]

edgeColumns = [c for c in edge_testData.columns if c not in ext_columns and \
                c not in DateColumns and c not in lag_columns and c != 'Hour']

edge_testData = edge_testData[edgeColumns]

edge_testData.shape

(2184, 257)

In [34]:
edge_prediction_df = edge_prediction_df[edge_testData.columns.astype(int)]
edge_prediction_df.head(2)

,1,10,100,101,102,106,107,108,109,11,...,90,91,92,93,94,95,96,97,98,99
0,0.392273,0.685473,5.062040,0.349090,0.446141,0.581170,4.773965,0.090140,0.006479,0.105381,...,3.022309,0.434057,3.177951,0.623091,0.119166,3.568820,0.056651,1.936740,0.727864,0.000775
1,0.002550,0.009726,0.032912,0.004953,0.016219,0.025169,0.281341,0.007914,-0.000905,0.009253,...,0.064435,0.030386,0.073939,0.014497,0.010295,0.083033,0.002060,0.083875,0.016935,-0.000108


In [35]:
r2_score(edge_testData.values, edge_prediction_df.values, multioutput='variance_weighted')

0.6677160019549366

In [36]:
# 300 epoch MSE bptt 12 hidden=100
#0.4597512505107641

In [37]:
# 150 epoch MSE bptt 12 hidden=100
# 0.47003788967204657

In [38]:
# 150 epoch L1 bptt 12 hidden=100
# 0.49459994974800275

In [39]:
# 300 epoch L1 bptt 12 hidden=100
# 0.48786324620679394

In [40]:
# 150 epoch L1 bptt 6 hidden=100
# 0.5141071457595932

In [41]:
# 150 epoch L1 bptt=6 hidden=50 
# 0.4721109077014878

In [42]:
# 150 epoch L1 bptt=6 hidden=200 
# 0.4574538111782432

In [43]:
# 300 epoch L1 bptt 6 hidden=100
# 0.5001768915084189

In [44]:
# 150 epoch L1 bptt 6 hidden=100 layers 2
# 0.5157145438811139

In [45]:
# 150 epoch L1 bptt 6 hidden=100 layers 3
# 0.5072893536554984